<a href="https://colab.research.google.com/github/kevinkmcguigan/cloud2cloud/blob/main/cloud_cloud_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

c2c2 = big time WIP

In [14]:
las_file_url = "https://download-telecharger.services.geo.ca/pub/elevation/pointclouds_nuagespoints/NRCAN/St_Johns_2020_2/NL_StJohns_20210205_NAD83CSRS_UTMZ22_1km_E2690_N52270_CQL1_CLASS.copc.laz"

In [ ]:
!git clone https://github.com/potree/PotreeConverter.git

In [ ]:
!wget https://github.com/Kitware/CMake/releases/download/v3.16.0/cmake-3.16.0-Linux-x86_64.sh
!chmod +x cmake-3.16.0-Linux-x86_64.sh
!./cmake-3.16.0-Linux-x86_64.sh --skip-license --prefix=/usr/local

In [ ]:
%cd PotreeConverter
!ls
!mkdir build
%cd build

In [ ]:
!cmake ../

In [ ]:
!make

In [15]:
import requests
import tempfile
# Download the LAZ file to a temporary file
with tempfile.NamedTemporaryFile(suffix=".laz", delete=False) as temp_laz_file:
    response = requests.get(las_file_url)
    temp_laz_file.write(response.content)
    input_file = temp_laz_file.name


In [ ]:
import subprocess
def run_potree_converter(input_file, output_directory, additional_args=None):
    # Define the path to the PotreeConverter executable
    potree_converter_path = '/content/PotreeConverter/build/PotreeConverter'
    
    # Define the command arguments
    args = [potree_converter_path, input_file, '-o', output_directory]
    
    # Add any additional arguments, if provided
    if additional_args:
        args.extend(additional_args)
    
    # Run the PotreeConverter command
    result = subprocess.run(args, capture_output=True, text=True)
    
    # Print the output of the PotreeConverter command
    print(result.stdout)
    print(result.stderr)
    
    return result.returncode

# Example usage
# input_file = 'path/to/pointcloud.las'
output_directory = '/content/potree_data'
additional_args = ['-p', '5000']  # Optional additional arguments
return_code = run_potree_converter(input_file, output_directory, additional_args)

if return_code == 0:
    print('PotreeConverter completed successfully.')
else:
    print('PotreeConverter encountered an error.')


In [25]:
#GPT4 attempt, with wrong metadata formatting
import os
import json
import numpy as np
import struct

def read_hierarchy_bin(file_path, node_size):
    with open(file_path, "rb") as f:
        nodes = []
        while True:
            node_name_data = f.read(node_size)
            if not node_name_data:
                break
            node_name = int.from_bytes(node_name_data, byteorder='little')
            point_count = int.from_bytes(f.read(4), byteorder='little')
            nodes.append((node_name, point_count))
        return nodes

def read_potree_bin(file_path, point_format, bbox_min, bbox_max):
    point_data = []

    def read_point(f):
        point = {}
        for attribute in point_format:
            point_value = struct.unpack(attribute["type"], f.read(attribute["size"]))[0]
            point[attribute["name"]] = point_value
        return point

    with open(file_path, "rb") as f:
        while True:
            point = read_point(f)
            coords = np.array([point["position"][0], point["position"][1], point["position"][2]])
            if np.all(bbox_min <= coords) and np.all(coords <= bbox_max):
                point_data.append(coords)

            if f.tell() == os.path.getsize(file_path):
                break

    return np.array(point_data)

def load_points_from_hierarchy(metadata_file, hierarchy_file, data_folder, bbox_min, bbox_max):
    with open(metadata_file, "r") as f:
        metadata = json.load(f)

    point_format = metadata["attributes"]
    node_size = len(metadata["stepSize"].encode('utf-8'))
    hierarchy_nodes = read_hierarchy_bin(hierarchy_file, node_size)
    point_data = []

    for node_name, point_count in hierarchy_nodes:
        if point_count > 0:
            file_path = os.path.join(data_folder, f"{node_name}.bin")
            points_in_node = read_potree_bin(file_path, point_format, bbox_min, bbox_max)
            if points_in_node.size > 0:
                point_data.append(points_in_node)

    return np.vstack(point_data) if point_data else np.array([])


In [ ]:
a = '/content/potree_data/pointclouds/5000/metadata.json'
b = '/content/potree_data/pointclouds/5000/hierarchy.bin'
c = '/content/potree_data/pointclouds/5000/octree.bin'
d = 0
e = 1000000000
data = load_points_from_hierarchy()

In [47]:
#GPT3.5 attempt with metadata.json as prompt
# never did get this to fully work to the point of producing a numpy array.
# this may work for potreeconverter 1.7 files as it seems to traverse that file structure 
import os
import json
import numpy as np
import struct

def read_potree_bin(file_path, point_format, bbox_min, bbox_max):
    point_data = []
    point_size = sum(attr["size"] for attr in point_format)
    
    def read_point(f):
        point = {}
        for attribute in point_format:
            point_value = struct.unpack(attribute["type"], f.read(attribute["size"]))[0]
            point[attribute["name"]] = point_value
        return point

    with open(file_path, "rb") as f:
        while True:
            point = read_point(f)
            coords = np.array([point["position"][0], point["position"][1], point["position"][2]])
            if np.all(bbox_min <= coords) and np.all(coords <= bbox_max):
                point_data.append(coords)

            if f.tell() == os.path.getsize(file_path):
                break

    return np.array(point_data)

def load_points_from_metadata(metadata_file, data_folder, bbox_min, bbox_max):
    with open(metadata_file, "r") as f:
        metadata = json.load(f)

    point_format = [attr for attr in metadata["attributes"] if attr["name"] == "position"]
    hierarchy = metadata["hierarchy"]
    first_chunk_size = hierarchy["firstChunkSize"]
    step_size = hierarchy["stepSize"]
    depth = hierarchy["depth"]
    point_data = []

    def traverse_octree(node_name, current_depth):
        if current_depth == depth:
            file_path = os.path.join(data_folder, f"{node_name}.bin")
            if os.path.exists(file_path):
                points_in_node = read_potree_bin(file_path, point_format, bbox_min, bbox_max)
                # print(points_in_node)
                if points_in_node.size > 0:
                    point_data.append(points_in_node)
            return

        for i in range(8):
            child_node_name = f"{node_name}{i}"
            # print(child_node_name)
            traverse_octree(child_node_name, current_depth + 1)

    traverse_octree("r", 0)
    return np.vstack(point_data) if point_data else np.array([])



In [48]:
# Example usage
metadata_file = '/content/potree_data/pointclouds/5000/metadata.json'
data_folder = '/content/potree_data/pointclouds/5000'
bbox_min = np.array([269100, 5227100, 100])
bbox_max = np.array([269200, 5227200, 200])
points_in_bbox = load_points_from_metadata(metadata_file, data_folder, bbox_min, bbox_max)


In [ ]:
points_in_bbox

In [62]:
import os
import json
import numpy as np
import struct

def read_hierarchy_bin(file_path, step_size):
    with open(file_path, "rb") as f:
        nodes = []
        while True:
            node_name_data = f.read(step_size)
            if not node_name_data:
                break
            node_name = int.from_bytes(node_name_data, byteorder='little')
            point_count = int.from_bytes(f.read(4), byteorder='little')
            nodes.append((node_name, point_count))
        return nodes


def read_octree_bin(file_path, point_format, hierarchy, bbox_min, bbox_max):
    point_data = []
    point_size = sum(attr["size"] for attr in point_format)
    
    def read_point(f):
        point = {}
        for attribute in point_format:
            point_value = struct.unpack(attribute["type"], f.read(attribute["size"]))[0]
            point[attribute["name"]] = point_value
        return point

    with open(file_path, "rb") as f:
        for node_name, point_count in hierarchy:
            for _ in range(point_count):
                point = read_point(f)
                coords = np.array([point["position"][0], point["position"][1], point["position"][2]])
                if np.all(bbox_min <= coords) and np.all(coords <= bbox_max):
                    point_data.append(coords)

    return np.array(point_data)

def load_points_from_hierarchy(metadata_file, hierarchy_file, octree_file, bbox_min, bbox_max):
    with open(metadata_file, "r") as f:
        metadata = json.load(f)

    point_format = [attr for attr in metadata["attributes"] if attr["name"] == "position"]
    hierarchy_step_size = metadata["hierarchy"]["stepSize"]
    hierarchy_nodes = read_hierarchy_bin(hierarchy_file, hierarchy_step_size)
    print(hierarchy_nodes)
    point_data = read_octree_bin(octree_file, point_format, hierarchy_nodes, bbox_min, bbox_max)

    return point_data


In [ ]:
metadata_file = '/content/potree_data/pointclouds/5000/metadata.json'
hierarchy_file = '/content/potree_data/pointclouds/5000/hierarchy.bin'
octree_file = '/content/potree_data/pointclouds/5000/octree.bin'
bbox_min = np.array([269100, 5227100, 100])
bbox_max = np.array([269200, 5227200, 200])
points_in_bbox = load_points_from_hierarchy(metadata_file, hierarchy_file, octree_file, bbox_min, bbox_max)


In [ ]:
#ive ran out of gpt4 tokens and 3.5 is going off the rails. ill have to start again later i think it is getting close.

In [ ]:
!pip install -q pypotree --upgrade

In [249]:
#modified from pypotree
port = 0
def display_cloud_colab(xyz, debug=False):
	"""
	Display a point cloud in the path generated by generate_cloud_for_display
	Arguments:
		xyz: a Nx3 matrix containing the 3D positions of N points
	"""

	global port

	if port == 0: 
		import portpicker
		import threading
		import socket
		import IPython

		from six.moves import socketserver
		from six.moves import SimpleHTTPServer

		import time

		class V6Server(socketserver.TCPServer):
			address_family = socket.AF_INET6

		class Handler(SimpleHTTPServer.SimpleHTTPRequestHandler):
			def do_GET(self):
				#super().do_GET()
				self.send_response(200)
				# If the response should not be cached in the notebook for
				# offline access:
				self.send_header('x-colab-notebook-cache-control', 'no-cache')
				self.end_headers()
				
				#self.wfile.write(b'''hola!''')
				self.wfile.write(b'''
				    document.querySelector('#output-area').appendChild(document.createTextNode('Script result!'));
                    ''')
		#global port
		port = portpicker.pick_unused_port()

		### SERVING ALL THE FILES
		Handler = SimpleHTTPServer.SimpleHTTPRequestHandler

		def server_entry():
			httpd = V6Server(('::', port), Handler)
			# Handle a single request then exit the thread.
			httpd.serve_forever()

		thread = threading.Thread(target=server_entry)
		thread.start()
  
		print ("server on port {}: thread {} ".format( port, thread ) )


	# text = open('point_clouds/{}.html'.format(xyz) ).read()
	text = open(xyz).read()

	# pointcloudpath='https://localhost:{port}/point_clouds/pointclouds/{xyz}'.format(port=port, xyz=xyz)
	pointcloudpath='https://localhost:{port}/{xyz}'.format(port=port, xyz=xyz)

	# print(pointcloudpath)
	newtext = text
	
  
	# newtext = newtext.replace('"./libs/potree/potree.js', '"https://localhost:{port}/pypotree/src/potree.colab.js'.format(port=port) )
  # newtext = newtext.replace('"./libs/potree/potree.js', '"https://raw.githubusercontent.com/centreborelli/pypotree/master/src/potree.colab.js' )
	newtext = newtext.replace('"./', '"https://localhost:{port}/potree_data/'.format(port=port))

	# newtext = text.replace('src="', 'src="https://localhost:{port}/'.format(port=port))
	# newtext = newtext.replace('href="', 'href="https://localhost:{port}/'.format(port=port))
	#newtext = newtext.replace('"pointclouds/', '"https://localhost:{port}/pointclouds/'.format(port=port))
	# newtext = newtext.replace(xyz, pointcloudpath)

	newtext = newtext.replace('width: 100%; height: 100%;', 'width: 100%; height: 500px;')
	# newtext = newtext.replace('libs/potree/potree.js', 'libs/potree/potree.colab.js' )
	
	if debug:
		print(newtext)

	
	else:
		import IPython
		return IPython.display.HTML(newtext)
 

 # soooo close to working....
 #something is wrong where the actual point cloud is not drawing. 
 # perhaps the path is incorrect or the pypotree.colab.js is really needed...

In [ ]:
!git clone https://github.com/centreborelli/pypotree.git

In [ ]:
# import pypotree
# %cd /content/
# !ls
# pypotree.display_cloud_colab('/content/potree_data/5000.html')
!ls
display_cloud_colab('/content/potree_data/5000.html', debug=True) #turn off debug to show the html

In [ ]:
open('/content/potree_data/5000.html').read()

In [ ]:
from IPython.display import HTML

# Define the path to the local HTML file
html_file_path = '/content/potree_data/5000.html'

# Read the contents of the HTML file
with open(html_file_path, "r") as f:
    html_content = f.read()

# Display the HTML content
HTML(html_content)

In [ ]:
from IPython.display import HTML

# Define a simple HTML snippet
html_content = "<h1>Hello, World!</h1>"

# Display the HTML content
HTML(html_content)